# Web Scraping for Indeed.com and Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [40]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [41]:
import requests
import bs4
from bs4 import BeautifulSoup

In [44]:
r = requests.get(URL)
soup = BeautifulSoup(r.content)
# Append to the full set of results
results = soup.findAll('div', { "class" : "result" })

/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some of the more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [75]:
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results: the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try different city). The second controls where in the results to start and gives 10 results (so we can keep incrementing this by 10 to move further within the list).

#### Complete the following code to collect results from multiple cities and start points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [1]:
YOUR_CITY = 'Atlanta'

In [74]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        r = requests.get(url_template.format(city, start))
        # Grab the results from the request (as above)
        soup = BeautifulSoup(r.content)
        # Append to the full set of results
        results += soup.findAll('div', { "class" : "result" })

/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [76]:
rows = []
for result in results:
    if result:
        row = {}
        row['title'] = get_title_from_result(result)
        row['company'] = get_company_from_result(result)
        row['summary'] = get_summary_from_result(result)
        row['salary'] = get_salary_from_result(result)
        row['city'] = city
        rows.append(row)

import pandas as pd
data = pd.DataFrame.from_records(rows)

data.head()

,city,company,salary,summary,title
0,Seattle,Aetna,None,Exceptional data science related experience wi...,Senior Manager - Senior Data Scientist
1,Seattle,Lancer Insurance Company,None,Lancer Insurance Company is looking for a tale...,Data Scientist & Analytics Developer
2,Seattle,Eli Lilly,None,Deliver next generation data integration and a...,IT Research Scientist
3,Seattle,"Amazon Web Services, Inc.",None,Interested in creating new state-of-the-art so...,Data Scientist - NYC
4,Seattle,ShopKeep,None,"We have a mature data infrastructure, and we’d...",Data Scientist


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [78]:
# Filter to only the rows that have salary entries
data = data[data.salary.notnull()]

# Remove duplicates
data = data.drop_duplicates()

# Filter out salary entries referring to week, hour or month
data = data[~(data.salary.astype('str').str.contains('hr'))]
data = data[~(data.salary.astype('str').str.contains('hour'))]
data = data[~(data.salary.astype('str').str.contains('week'))]
data = data[~(data.salary.astype('str').str.contains('wk'))]
data = data[~(data.salary.astype('str').str.contains('month'))]

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [73]:
import re
import numpy as np
def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])

In [79]:
data['parsed_salary'] = data['salary'].map(extract_salary_average)

/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


#### Save the scraped results as a CSV

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [7]:
import pandas as pd
salary_data = pd.read_csv('../assets/indeed-scraped-job-postings.csv')
salary_data.head()

,city,company,salary,summary,title,parsed_salary
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [8]:
median_salary = salary_data.parsed_salary.median()
salary_data['HighSalary'] = (salary_data['parsed_salary'] > median_salary).astype(int)

#### Thought experiment: What is the baseline accuracy for this model?

In [9]:
# It is 50% if we guess randomly, half the salaries will be below the median and half will be above.

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [10]:
import statsmodels.formula.api as sm

model = sm.logit("HighSalary ~ city", data=salary_data).fit()
model.summary()

         Current function value: 0.593851
         Iterations: 35


/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             HighSalary   No. Observations:                  413
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                           17
Date:                Mon, 11 Jul 2016   Pseudo R-squ.:                  0.1422
Time:                        01:11:18   Log-Likelihood:                -245.26
converged:                      False   LL-Null:                       -285.92
                                        LLR p-value:                 2.234e-10
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                -1.3218      0.563     -2.349      0.019        -2.425    -0.219
city[T.Austin]            1.0341      0.949      1.090      0.276        -0.825     2.893
city[T.Boston]            2.0794      0.644      3.230      0.001         0.817     3.341
city[T.Chicago]           1.4330      0.654      2.190      0.029         0.151     2.715
city[T.Dallas]            0.6286      0.832      0.756      0.450        -1.001     2.259
city[T.Denver]           -0.2187      0.849     -0.257      0.797        -1.883     1.446
city[T.Houston]         -22.8306   5.85e+04     -0.000      1.000     -1.15e+05  1.15e+05
city[T.Los+Angeles]       1.3218      0.695      1.901      0.057        -0.041     2.684
city[T.Miami]            -0.2877      1.232     -0.234      0.815        -2.701     2.126
city[T.New+York]          0.9886      0.597      1.656      0.098        -0.182     2.159
city[T.Palo+Alto]         2.1868      0.703      3.110      0.002         0.809     3.565
city[T.Philadelphia]      1.8814      0.842      2.234      0.026         0.230     3.532
city[T.Phoenix]         -24.0875   1.34e+05     -0.000      1.000     -2.63e+05  2.63e+05
city[T.Pittsburgh]      -22.8306   5.85e+04     -0.000      1.000     -1.15e+05  1.15e+05
city[T.Portland]        -15.4028   4282.475     -0.004      0.997     -8408.899  8378.093
city[T.San+Diego]         0.9163      0.856      1.070      0.285        -0.762     2.595
city[T.San+Francisco]     2.4478      0.653      3.746      0.000         1.167     3.729
city[T.Seattle]           1.3218      0.719      1.839      0.066        -0.087     2.731
=========================================================================================
"""

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [11]:
salary_data['is_senior'] = salary_data['title'].str.contains('Senior').astype(int)
salary_data['is_director'] = salary_data['title'].str.contains('Director').astype(int)
salary_data['is_manager'] = salary_data['title'].str.contains('Manager').astype(int)

model = sm.logit("HighSalary ~ city + is_senior + is_manager", data=salary_data).fit()
model.summary()


         Current function value: 0.583474
         Iterations: 35


/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             HighSalary   No. Observations:                  413
Model:                          Logit   Df Residuals:                      393
Method:                           MLE   Df Model:                           19
Date:                Mon, 11 Jul 2016   Pseudo R-squ.:                  0.1572
Time:                        01:11:23   Log-Likelihood:                -240.97
converged:                      False   LL-Null:                       -285.92
                                        LLR p-value:                 3.469e-11
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                -1.3182      0.563     -2.340      0.019        -2.422    -0.214
city[T.Austin]            0.8884      0.959      0.926      0.354        -0.992     2.769
city[T.Boston]            1.9411      0.647      3.001      0.003         0.673     3.209
city[T.Chicago]           1.3342      0.657      2.030      0.042         0.046     2.622
city[T.Dallas]            0.4513      0.842      0.536      0.592        -1.199     2.102
city[T.Denver]           -0.2104      0.851     -0.247      0.805        -1.879     1.458
city[T.Houston]         -21.6234   2.74e+04     -0.001      0.999     -5.36e+04  5.36e+04
city[T.Los+Angeles]       1.2439      0.698      1.783      0.075        -0.124     2.611
city[T.Miami]            -0.6919      1.260     -0.549      0.583        -3.162     1.778
city[T.New+York]          0.8731      0.599      1.456      0.145        -0.302     2.048
city[T.Palo+Alto]         2.0951      0.706      2.970      0.003         0.712     3.478
city[T.Philadelphia]      1.7234      0.850      2.028      0.043         0.057     3.389
city[T.Phoenix]         -24.0910   1.34e+05     -0.000      1.000     -2.63e+05  2.63e+05
city[T.Pittsburgh]      -21.8127   3.23e+04     -0.001      0.999     -6.33e+04  6.32e+04
city[T.Portland]        -16.4545   4423.749     -0.004      0.997     -8686.844  8653.935
city[T.San+Diego]         0.4993      0.884      0.565      0.572        -1.234     2.232
city[T.San+Francisco]     2.2527      0.658      3.425      0.001         0.964     3.542
city[T.Seattle]           1.2322      0.723      1.705      0.088        -0.184     2.649
is_senior                 0.9833      0.352      2.790      0.005         0.293     1.674
is_manager               -0.0682      0.463     -0.147      0.883        -0.976     0.840
=========================================================================================
"""

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
model = LogisticRegression(penalty = 'l2', C=0.1)

from patsy import dmatrix
X = dmatrix("city + is_senior + is_manager + 0", data=salary_data) # Remember the + 0  to remove the Intercept term
X_scaled = scaler.fit_transform(X)
y = salary_data['HighSalary']

model.fit(X_scaled, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [13]:
from sklearn.cross_validation import cross_val_score

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

accuracy 0.532740928806 0.0166458939697
precision 0.519592055824 0.0302787005393
recall 0.252525252525 0.0894951775084
roc_auc 0.513385023948 0.0713438440657


#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [14]:
model = LogisticRegression(penalty = 'l1', C=1.0)

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

print()    
model = LogisticRegression(penalty = 'l2', C=1.0)

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())
    
print()
model = LogisticRegression(penalty = 'l1', C=0.1)

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())


accuracy 0.547233682429 0.0242551603251
precision 0.59012345679 0.105683627452
recall 0.247474747475 0.0936729140959
roc_auc 0.475290364522 0.0560828230827

accuracy 0.532740928806 0.0166458939697
precision 0.514653784219 0.0366329563847
recall 0.222222222222 0.114947542189
roc_auc 0.513385023948 0.0713438440657

accuracy 0.561726436052 0.00770585416491
precision 0.635978835979 0.0735582950137
recall 0.247474747475 0.0936729140959
roc_auc 0.534037558685 0.00682020167896


In [16]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : X.design_info.column_names, 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)
df

,coef,features
16,0.309884,city[San+Francisco]
2,0.205423,city[Boston]
18,0.182683,is_senior
10,0.165924,city[Palo+Alto]
11,0.020842,city[Philadelphia]
3,0.000772,city[Chicago]
1,0.000000,city[Austin]
17,0.000000,city[Seattle]
15,0.000000,city[San+Diego]
19,0.000000,is_manager


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are most valuable? 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(salary_data.summary)
X_scaled = scaler.fit_transform(X.toarray())

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

accuracy 0.658485842237 0.0436273832017
precision 0.694949494949 0.0768739296583
recall 0.525252525253 0.0142849854785
roc_auc 0.722178257536 0.0352551146836


/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [18]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : vectorizer.get_feature_names(), 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)

In [19]:
df.head()

,coef,features
1454,0.276842,science
1457,0.229212,scientist
921,0.226615,leading
1047,0.223824,models
923,0.223101,learning


In [20]:
df.tail()

,coef,features
1318,-0.079242,provide
1304,-0.108528,project
104,-0.113453,and
1383,-0.137474,related
1408,-0.160300,research


#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients

In [21]:
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV()

X = vectorizer.fit_transform(salary_data.summary)
X_scaled = scaler.fit_transform(X.toarray())

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

accuracy 0.702175676152 0.00102229902425
precision 0.723376515299 0.031149832171
recall 0.621212121212 0.0566917785875
roc_auc 0.7708679123 0.0210257205031


/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/arahuja/anaconda/envs/py34/lib/python3.4/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
